In [8]:
#import necessary packages
import os
import re
import string
import time
from datetime import date,datetime, timezone,timedelta
import logging
from logging.handlers import TimedRotatingFileHandler
import io
import pandas as pd
import joblib
import numpy  as np
from sklearn.svm import LinearSVC
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('popular')
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context




In [3]:
def preprocess_text(message):

    #stopwords
    stpwrd = nltk.corpus.stopwords.words('english')
    #initialize lemmatizing 
    lemmatizer = WordNetLemmatizer()
    #lowering input
    message=message.lower()
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message)
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    #removing hyperlinks
    message = re.sub(r'http\S+', ' ', message)   
    return message

In [5]:
model = joblib.load('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/code/finalized_model.sav')



In [15]:
df=pd.read_csv('/Users/jghosh2/Downloads/NONINCA_QB1_29Jun2023_131621-selected/Input_RawData_29Jun2023_132529.csv')

# dedup version

In [16]:
print(df.shape)
#combining input features to a single column
df['combined_text'] = df[['item_name','description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
# apply data preprocessing steps on the prepared column
df['processed_text']= df['combined_text'].map(lambda s:preprocess_text(s))
print(df.shape)

df['item_name_lower'] = df['item_name'].str.lower()
df['description_lower'] = df['description'].str.lower()
df['establishment_type_lower'] = df['establishment_type'].str.lower()
print(df.shape)

#removing duplicates
dedup_df=df.drop_duplicates(subset=['item_name_lower','description_lower','establishment_type_lower'],ignore_index=True)


(552526, 17)
(552526, 19)
(552526, 19)


In [17]:
print(dedup_df.shape)
#getting ml predictions
dedup_df['ml_prediction'] = model.predict(dedup_df['processed_text'])
#combining cat name and integer column to get a single output column
dedup_df[['ml_predicted_cat_name','ml_predicted_integer']] = dedup_df['ml_prediction'].str.split(':', expand=True)
#getting confidence score
#dedup_df['ml_predicted_conf_score'] =np.round_(1/(1+np.exp(-np.round_(np.max(model.decision_function(dedup_df['processed_text'].values), axis=1), decimals=2))),decimals=2)
#dropping unnecessary column



(89231, 19)


<ipython-input-17-3fe6fcdad380>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dedup_df['ml_prediction'] = model.predict(dedup_df['processed_text'])
<ipython-input-17-3fe6fcdad380>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dedup_df[['ml_predicted_cat_name','ml_predicted_integer']] = dedup_df['ml_prediction'].str.split(':', expand=True)


In [18]:
dedup_df.drop(['merchant_uuid','store_name','parent_chain_uuid','parent_chain_name',
               'merchant_type_analytics','item_uuid','subsection_name','created_at_epoch_seconds',
               'item_name_lower','description_lower','establishment_type_lower',
               'processed_text','combined_text','ml_prediction'], inplace=True, axis=1)

<ipython-input-18-dfa1dee45917>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dedup_df.drop(['merchant_uuid','store_name','parent_chain_uuid','parent_chain_name',


In [19]:
dedup_df.shape

(89231, 6)

In [20]:
dedup_df[dedup_df['item_name']=='JALAPE√ëOS']

,establishment_type,item_name,description,ml_predicted_cat_name,ml_predicted_integer,ml_predicted_conf_score


In [26]:
duplicate_df=df[df.duplicated(subset=['item_name_lower','description_lower','establishment_type_lower'])]


In [27]:
duplicate_df.shape

(463295, 19)

In [29]:
duplicate_df[duplicate_df['item_name']=='JALAPE√ëOS']

,merchant_uuid,store_name,parent_chain_uuid,parent_chain_name,establishment_type,merchant_type_analytics,item_uuid,item_name,subsection_name,description,created_at_epoch_seconds,item_name_lower,description_lower,establishment_type_lower,ml_predicted_cat_name,ml_predicted_integer,ml_predicted_conf_score,combined_text,processed_text


In [31]:
552526-89231

463295

In [32]:
df[df['item_name']=='JALAPE√ëOS']


,merchant_uuid,store_name,parent_chain_uuid,parent_chain_name,establishment_type,merchant_type_analytics,item_uuid,item_name,subsection_name,description,created_at_epoch_seconds,item_name_lower,description_lower,establishment_type_lower,ml_predicted_cat_name,ml_predicted_integer,ml_predicted_conf_score,combined_text,processed_text


In [34]:
df[df['item_name']=='2 Pieces of Toast']


,merchant_uuid,store_name,parent_chain_uuid,parent_chain_name,establishment_type,merchant_type_analytics,item_uuid,item_name,subsection_name,description,created_at_epoch_seconds,item_name_lower,description_lower,establishment_type_lower,ml_predicted_cat_name,ml_predicted_integer,ml_predicted_conf_score,combined_text,processed_text
8,0001998e-2cfd-475d-9698-da9625849529,Village Inn (1919 North Harrison Street),bb0f3764-7721-4f14-b24d-cd11b407628e,Village Inn (Parent),DEFAULT,MERCHANT_TYPE_RESTAURANT,d0d43928-ed03-5630-99c2-d0f25d57b954,2 Pieces of Toast,A la Carte,Your choice of bread,1687877044,2 pieces of toast,your choice of bread,default,"CAT_PREPARED_FOOD,TEMP_HEATED","101,1",0.61,2 Pieces of Toast Your choice of bread DEFAULT,piece toast choice bread default


In [36]:
dedup_df.to_csv('dedup_df.csv')

In [37]:
duplicate_df.to_csv('duplicate_df.csv')